In [ ]:
# prompt: google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Getting token,headers

In [ ]:
import requests

# Spotify API 的 URL
url = "https://accounts.spotify.com/api/token"


client_id = "f90b66a42a91467c99fba6d8c5e4932d"
client_secret = "55aed84eedcc418fa5057d0b05a7d491"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}
# 发起 POST 请求以获取 token
response = requests.post(url, headers=headers, data=data)
# 解析 JSON 响应
if response.status_code == 200:
    access_token = response.json().get('access_token')
    print(f"Access Token: {access_token}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)
headers = {
    "Authorization": f"Bearer {access_token}"
}
# Spotify Show Episodes API URL
#show_id = "1iwcjlgreRD0nJxgtdwMLx"
#url = f"https://api.spotify.com/v1/shows/{show_id}/episodes"

#response = requests.get(url, headers=headers)

Access Token: BQAOov5rqf5oza9XijnPIQmrCvOU2uqGSVReZlMPLrsI53PNZgBB0WJGWXDVWyxk1MrdR-VEvP3LU4pWWNn3kjERFhmHRcfFy9c2mrNiSiZGvZ_zyAU


# Getting dataframe for the show (named data)

## Sample for getting one show for each category

In [ ]:
import pandas as pd
data = pd.DataFrame(columns=["Show Name", "Show ID", "Category"])

# 遍历每个类别关键词
# 遍历每个类别关键词
podcast_categories_keywords = ['arts', 'business', 'educational', 'entertainment', 'games', 'health',
                               'lifestyle', 'news', 'politics', 'recreation', 'sports', 'technology', 'true crime']
for keyword in podcast_categories_keywords:
    url = f"https://api.spotify.com/v1/search?q={keyword}&type=show&limit=20"  # 每个关键词获取一个节目
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data_json = response.json()

        # 提取节目列表
        shows = data_json.get("shows", {}).get("items", [])

        if shows:
            # 获取第一个节目的名称和ID
            show_name = shows[0].get("name", "No name available")
            show_id = shows[0].get("id", "No ID available")

            # 将数据添加到 DataFrame
            new_row = pd.DataFrame({"Show Name": [show_name], "Show ID": [show_id], "Category": [keyword]})
            data = pd.concat([data, new_row], ignore_index=True)
        else:
            print(f"No shows available for category: {keyword}")
    else:
        print(f"Error fetching shows for category {keyword}: {response.status_code}")

# 打印最终的 DataFrame
print(data)

                        Show Name                 Show ID       Category
0   Arts Educators Save the World  0OPE2gJZ58WoEpJ2klv7FY           arts
1             Business Breakdowns  417NPBWqtMbDU0FlWZTRDC       business
2           Stuff You Should Know  0ofXAdFIQQRsCYj9754UFx    educational
3                         E! News  1wAEG2evtfeXQy5GhceOta  entertainment
4                Games with Names  2XCaxqeyDJF9VrNW0ViPo3          games
5                  HealthyGamerGG  6VaJwyS2KXxiXqR77jqzmP         health
6                   咪一下 LifeStyle  4bakEXixuQK2IEq0oXlAsN      lifestyle
7                   The Headlines  0pymDUChxmrkH41tOT33Uc           news
8        The NPR Politics Podcast  2PnWnDNwU3wKesCHCpy252       politics
9    Real Talk Recreation Therapy  4fg3n1nzVjhFCuVJrBJyjk     recreation
10            The Pat McAfee Show  1ol0jP8hrNHydiAd38G6B1         sports
11                       TED Tech  1iwcjlgreRD0nJxgtdwMLx     technology
12     Murder: True Crime Stories  4fsW5D9rKYycsP2h

## Formal code for getting 100 shows each for each category.

In [ ]:
import requests
import pandas as pd
import re

podcast_categories_keywords = ['arts', 'business', 'educational', 'entertainment', 'games', 'health',
                               'lifestyle', 'news', 'politics', 'recreation', 'sports', 'technology', 'true crime']

# 初始化一个空的 DataFrame 来存储结果
data = pd.DataFrame(columns=["Show Name", "Show ID", "Category"])

# 正则表达式：匹配英文字符、数字、空格和常见的英文标点符号
english_characters_pattern = re.compile("^[A-Za-z0-9\s\.,;?!'-]*$")

def is_english(name):
    """检查节目名称是否只包含英文字符"""
    return bool(english_characters_pattern.match(name))

for keyword in podcast_categories_keywords:
    url = f"https://api.spotify.com/v1/search?q={keyword}&type=show&limit=20"
    english_shows = []  # 存储每个类别的英文节目

    while url and len(english_shows) < 100:  # 持续请求直到获取到200个节目或没有更多数据
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data_json = response.json()

            # 提取节目列表并筛选英文节目
            shows = data_json.get("shows", {}).get("items", [])
            filtered_shows = [show for show in shows if is_english(show.get("name", ""))]
            english_shows.extend(filtered_shows)

            # 如果还未达到200个并且有下一页，继续请求
            if len(english_shows) < 100 and data_json.get("shows", {}).get("next"):
                url = data_json["shows"]["next"]
            else:
                url = None  # 如果没有下一页数据，停止请求
        else:
            print(f"Error fetching shows for category {keyword}: {response.status_code}")
            break

    # 截取前 200 个节目
    english_shows = english_shows[:100]

    # 将每个节目存入 DataFrame
    for show in english_shows:
        show_name = show.get("name", "No name available")
        show_id = show.get("id", "No ID available")
        new_row = pd.DataFrame({"Show Name": [show_name], "Show ID": [show_id], "Category": [keyword]})
        data = pd.concat([data, new_row], ignore_index=True)

    print(f"Fetched {len(english_shows)} English shows for category: {keyword}")

# 打印最终的 DataFrame
print(data)

# 如果需要保存数据，可以使用如下代码
data.to_csv("/content/drive/MyDrive/628Mod4/podcast_shows.csv", index=False)


Fetched 100 English shows for category: arts
Fetched 100 English shows for category: business
Fetched 100 English shows for category: educational
Fetched 100 English shows for category: entertainment
Fetched 100 English shows for category: games
Fetched 100 English shows for category: health
Fetched 100 English shows for category: lifestyle
Fetched 100 English shows for category: news
Fetched 100 English shows for category: politics
Fetched 100 English shows for category: recreation
Fetched 100 English shows for category: sports
Fetched 100 English shows for category: technology
Fetched 100 English shows for category: true crime
                          Show Name                 Show ID    Category
0     Arts Educators Save the World  0OPE2gJZ58WoEpJ2klv7FY        arts
1                      Distractible  2X40qLyoj1wQ2qE5FVpA7x        arts
2                              Arts  2lcrCfm2uIaQzAYLg8dGWu        arts
3                     Modern Wisdom  0XrOqvxlqQI6bmdYHuIVnr        arts
4  

#Getting episode data according to Show ID

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm  # 导入进度条库
import time  # 导入时间模块，用于休息一秒

# 定义函数：请求每个 show 的所有 episodes 数据（最多20集）
def fetch_episodes(show_id, show_name, category):
    url = f"https://api.spotify.com/v1/shows/{show_id}/episodes?limit=20"
    episodes_data = []

    # 获取最多20集数据，单次请求就足够
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()

        # 提取 episodes 列表
        episodes = data.get("items", [])

        # 遍历每个 episode，提取相关信息
        for episode in episodes:
            if episode is None:
                print(f"Error: Found a None value in episodes for show {show_name} ({show_id})")
                continue

            # 确保 episode 是字典类型
            if isinstance(episode, dict):
                episode_name = episode.get("name", "No name available")
                episode_id = episode.get("id", "No ID available")
                description = episode.get("description", "No description available")
                duration_ms = episode.get("duration_ms", 0)
                duration_min = duration_ms / 60000  # 转换为分钟
                release_date = episode.get("release_date", "No release date available")

                # 将信息添加到列表中
                episodes_data.append({
                    "Episode Name": episode_name,
                    "Episode ID": episode_id,
                    "Description": description,
                    "Release Date": release_date,
                    "Duration (minutes)": duration_min,
                    "Show Name": show_name,
                    "Show ID": show_id,
                    "Category": category
                })
            else:
                print(f"Error: Skipping invalid episode data for show {show_name} ({show_id})")

    else:
        print(f"Error fetching episodes for show {show_name} ({show_id}): {response.status_code}")

    # 请求后休息一秒
    time.sleep(1)  # 延时1秒

    return episodes_data

# 初始化一个空的列表，用于存储所有节目数据
all_episodes_data = []

# 假设 data 是你存储播客数据的 DataFrame，其中包含 Show ID、Show Name 和 Category 列
# 例如: data = pd.read_csv('some_file.csv')
data = pd.read_csv("/content/drive/MyDrive/628Mod4/podcast_shows.csv")
# 遍历不同类别的数据
categories = data['Category'].unique()  # 获取所有类别

for category_index, category in enumerate(categories, start=1):
    print(f"\nStarting to fetch data for category {category_index}/{len(categories)}: {category}")

    category_data = data[data['Category'] == category]

    # 使用 tqdm 来展示进度条
    for show_index, row in tqdm(category_data.iterrows(), total=category_data.shape[0], desc=f"Category {category}"):
        show_id = row['Show ID']
        show_name = row['Show Name']

        # 获取每个 show 的前20集数据
        episodes = fetch_episodes(show_id, show_name, category)

        # 将获取到的 episodes 数据添加到列表中
        all_episodes_data.extend(episodes)

    # 打印该类别处理完毕的信息
    print(f"Finished fetching data for category {category} ({category_index}/{len(categories)})")

# 将所有的数据转换成 DataFrame
episodes_df = pd.DataFrame(all_episodes_data)

# 打印 DataFrame 前几行
print(episodes_df.head())
print(f"Total records fetched: {episodes_df.shape[0]}")

# 将 DataFrame 保存为 CSV 文件
episodes_df.to_csv("/content/drive/MyDrive/628Mod4/podcast_episodes.csv", index=False, encoding="utf-8")
print("Data fetching complete, saved to 'podcast_episodes.csv'")



Starting to fetch data for category 1/13: arts


Category arts: 100%|██████████| 100/100 [01:54<00:00,  1.14s/it]


Finished fetching data for category arts (1/13)

Starting to fetch data for category 2/13: business


Category business:  23%|██▎       | 23/100 [00:26<01:27,  1.14s/it]

Error fetching episodes for show Fox Business Hourly Report (44oRTc7X5qXIqtTax9GZbS): 404


...

Error: Found a None value in episodes for show True Crime New England (31gk7vHwSmFuMwT15i51If)


Category true crime:  98%|█████████▊| 98/100 [01:50<00:02,  1.12s/it]

Error: Found a None value in episodes for show Historical True Crime  (5EoajP8947TJKmjEsCy5o8)


Category true crime: 100%|██████████| 100/100 [01:52<00:00,  1.12s/it]


Finished fetching data for category true crime (13/13)
                                        Episode Name              Episode ID  \
0                  Cody Renard Richard & Carrie Wood  5uXBfMhL9rbUKQaZLQZaFD   
1                     Carlos Gacharna & Geof Hermann  7dCEFEbrvs7Efr4DriVBoZ   
2                       Jonathan Groff & Linda Tough  409iSTKN6X37pI9aOpVeKc   
3                                       WE'RE BACK!!  5xBZeMzgoDIK95ad8HwpmS   
4  Top Chef's Valentine Howell, Jr. with... his m...  3qfyXzLyCD8rv9twjH5qYF   

                                         Description Release Date  \
0  Broadway and TV's finest production stage mana...   2024-11-30   
1  Multimedia artist, teaching artist, and all ar...   2024-11-21   
2  Broadway's reigning Tony-award winner for Best...   2024-11-14   
3  Erica and Alek are back! And from now on, we'r...   2024-11-07   
4  So many first-time ingredients to spice up the...   2024-04-02   

   Duration (minutes)                      Show N

# Check downloaded data

In [ ]:
# prompt: episodes_df count by 'Category'

import pandas as pd

# Load the dataframe
episodes_df = pd.read_csv("/content/drive/MyDrive/628Mod4/podcast_episodes2.csv")

# Count by category
category_counts = episodes_df.groupby('Category').size()
print(category_counts)
# 筛选出计数小于20的节目
# category_counts_less_than_20 = category_counts[category_counts < 20]

# 打印结果
# print(category_counts_less_than_20)


Category
arts             1875
business         1884
educational      1857
entertainment    1910
games            1747
health           1674
lifestyle        1763
news             1657
politics         1848
recreation       1884
sports           1491
technology       1661
true crime       1635
dtype: int64


In [ ]:
# 1. 按 'Show Name' 计算每个播客的 episode 数量
category_counts = episodes_df.groupby('Show Name').size()

# 2. 找出那些 episode 数量小于 20 的播客
shows_to_remove = category_counts[category_counts < 20].index

# 3. 从 episodes_df 中删除这些播客对应的所有行
episodes_df_cleaned = episodes_df[~episodes_df['Show Name'].isin(shows_to_remove)]

# 打印清理后的 DataFrame
# print(episodes_df_cleaned)


In [ ]:
episodes_df_cleaned.groupby('Show Name').size().unique()

array([ 20,  40,  43,  60,  31,  38,  49,  39,  37, 140,  80,  65,  55,
        30,  28, 100,  21])

In [ ]:
# 1. 按 'Show Name' 分组并计算每个播客的 episode 数量
show_sizes = episodes_df_cleaned.groupby('Show Name').size()

# 2. 获取 episode 数量为 31 的播客名称
shows_with_31_episodes = show_sizes[show_sizes == 38].index

# 3. 从 episodes_df_cleaned 中筛选出这些播客的信息
filtered_episodes_df = episodes_df_cleaned[episodes_df_cleaned['Show Name'].isin(shows_with_31_episodes)]

# 打印筛选后的结果
print(filtered_episodes_df)


                                            Episode Name  \
10426    How a Nuclear Lab Helped Catch a Serial Killer    
10427     The Pill: Is It Messing Up Our Brain and Body?   
10428  Lucid Dreaming: The Bonkers World Inside Our M...   
10429                 Aphantasia: Missing the Mind's Eye   
10430     Fiber: The Secret to a Healthier, Happier You?   
...                                                  ...   
19334  HIGHLIGHT: Upset Alert & Nick's Picks | NFL We...   
19335  Giants release Daniel Jones, Steelers fall to ...   
19336  HIGHLIGHT: Broussard's Under Duress List | NFL...   
19337  Kyle Shanahan defends Brock Purdy, Davante Ada...   
19338  WHAT'S WRIGHT: Jets Season of Chaos, Josh Over...   

                   Episode ID  \
10426  383sunWBthuLTikfu7uRyJ   
10427  74VASCzv5IWDKWfxmmY4Z2   
10428  1ZutPYpYnOn1PulAnPm9Ct   
10429  6bKBBSfSEuhTneMeLiHaoW   
10430  5y7Hma4j4DlQJ2ey4655wI   
...                       ...   
19334  5QLwdcXF2tXOR0VBHlOQnW   
19335  0wuqSJBw

In [ ]:
episodes_df_cleaned.to_csv("/content/drive/MyDrive/628Mod4/podcast_episodes.csv", index=False, encoding="utf-8")

# Data cleaning

In [ ]:
import pandas as pd
episodes_df_cleaned = pd.read_csv("/content/drive/MyDrive/628Mod4/podcast_episodes.csv")
episodes_df_cleaned.head()

,Episode Name,Episode ID,Description,Release Date,Duration (minutes),Show Name,Show ID,Category
0,Cody Renard Richard & Carrie Wood,5uXBfMhL9rbUKQaZLQZaFD,Broadway and TV's finest production stage mana...,2024-11-30,61.728217,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts
1,Carlos Gacharna & Geof Hermann,7dCEFEbrvs7Efr4DriVBoZ,"Multimedia artist, teaching artist, and all ar...",2024-11-21,62.517100,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts
2,Jonathan Groff & Linda Tough,409iSTKN6X37pI9aOpVeKc,Broadway's reigning Tony-award winner for Best...,2024-11-14,66.047550,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts
3,WE'RE BACK!!,5xBZeMzgoDIK95ad8HwpmS,"Erica and Alek are back! And from now on, we'r...",2024-11-07,1.462850,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts
4,"Top Chef's Valentine Howell, Jr. with... his m...",3qfyXzLyCD8rv9twjH5qYF,So many first-time ingredients to spice up the...,2024-04-02,61.217083,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts


In [ ]:
#url
#punctuation
#lower
#tokenize/stopwords

## Remove url, punctuation and lower them

In [ ]:
import re
import pandas as pd

def clean_description(df):
    url_pattern = r'https?://\S+|www\.\S+'
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'
    punctuation_pattern = r'[^\w\s]'

    def clean_text(text):

        text = str(text)

        text = re.sub(url_pattern, '', text)

        text = re.sub(email_pattern, '', text)

        text = re.sub(punctuation_pattern, '', text)

        text = text.lower()
        return text

    df['Description'] = df['Description'].apply(clean_text)
    return df

In [ ]:
df = episodes_df_cleaned
df = clean_description(df)

## Exclude stop words

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

nltk.download('punkt_tab')

def remove_stopwords(df):

    stop_words = set(stopwords.words('english'))

    def clean_text(text):
        # If the text is missing, return an empty string
        if not text:
            return ""
        # Tokenize the text
        words = word_tokenize(text.lower())  # Convert text to lowercase first
        # Filter out stopwords
        filtered_words = [word for word in words if word not in stop_words]
        # Rejoin into a single string
        return ' '.join(filtered_words)

    # Apply the cleaning function to the DataFrame's Description column
    df['Description'] = df['Description'].apply(clean_text)
    return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from nltk.corpus import stopwords

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Print the stopwords
print(stop_words)

{"wasn't", 'i', "won't", 'ourselves', 'y', 'both', 'aren', "aren't", 'mustn', 'after', 'been', 'into', 'so', 'weren', 'do', "don't", 'have', 'against', 'in', 'their', 'these', 'shouldn', 'you', 'he', 'below', 'off', "shan't", 's', 'she', 'over', 'was', 'as', 'your', 'ours', 'wouldn', 'me', 'didn', 'what', 'are', 've', "hasn't", 'him', 'will', 'out', 'more', 'too', 'does', 'own', 'once', "isn't", "you'd", "she's", 'did', 'here', 'should', 'than', "wouldn't", 'from', 'at', 'her', 'by', 'our', "that'll", 'for', 'each', 'because', 'can', 'itself', 'am', 'why', 'isn', 'herself', 'hers', 'just', 'needn', 'doing', 'theirs', 'such', 'further', 'until', 'other', 'his', "doesn't", 'mightn', 'where', 'wasn', 're', 'between', "shouldn't", 'yours', 'of', 'through', 'how', 'we', 'no', "didn't", 'couldn', "haven't", "it's", 'most', 'o', 'hasn', 'all', "mustn't", 'there', "you'll", 'll', 'who', 't', 'with', 'which', 'whom', 'few', 'again', 'they', 'm', 'an', 'having', 'and', 'themselves', 'above', 'd'

In [ ]:
cleaned_df = remove_stopwords(df)
cleaned_df.head(3)

,Episode Name,Episode ID,Description,Release Date,Duration (minutes),Show Name,Show ID,Category
0,Cody Renard Richard & Carrie Wood,5uXBfMhL9rbUKQaZLQZaFD,broadway tvs finest production stage manager c...,2024-11-30,61.728217,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts
1,Carlos Gacharna & Geof Hermann,7dCEFEbrvs7Efr4DriVBoZ,multimedia artist teaching artist around badas...,2024-11-21,62.517100,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts
2,Jonathan Groff & Linda Tough,409iSTKN6X37pI9aOpVeKc,broadways reigning tonyaward winner best actor...,2024-11-14,66.047550,Arts Educators Save the World,0OPE2gJZ58WoEpJ2klv7FY,arts


In [ ]:
import pandas as pd
from collections import Counter

def word_frequency_by_category(df, category):

    filtered_df = df[df['Category'] == category]

    all_words = ' '.join(filtered_df['Description']).split()

    word_counts = Counter(all_words)

    word_freq_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency'])
    word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

    return word_freq_df

pd.set_option('display.max_rows', 30)
result_df = word_frequency_by_category(cleaned_df, 'business')
print(result_df[1:30])


           Word  Frequency
1       episode       1428
2           get       1078
3       podcast        969
4          free        929
5            us        864
6         learn        843
7           new        807
8          show        711
9   information        494
10         help        491
11    financial        460
12         like        451
13         also        447
14        visit        432
15          tax        413
16    instagram        408
17          one        405
18   investment        403
19         make        400
20        first        391
21        click        387
22        today        386
23         time        378
24        youre        372
25         life        367
26   strategies        366
27         want        360
28        start        359
29        money        354


In [ ]:
import pandas as pd
from collections import Counter
import numpy as np

def word_frequency_by_category(df):

    all_word_freq = []

    for category, group in df.groupby('Category'):
        all_words = ' '.join(group['Description']).split()
        word_counts = Counter(all_words)

        for word, count in word_counts.items():
            all_word_freq.append([word, count, category])

    word_freq_df = pd.DataFrame(all_word_freq, columns=['Word', 'Frequency', 'Category'])
    return word_freq_df
def calculate_representativeness(df, word_freq_df):

    category_totals = df.groupby('Category')['Description'].apply(lambda x: ' '.join(x).split()).apply(len)

    total_words_all_categories = df['Description'].apply(lambda x: ' '.join(x).split()).apply(len).sum()

    all_word_counts = Counter(' '.join(df['Description']).split())

    word_representativeness = []

    for _, row in word_freq_df.iterrows():
        word = row['Word']
        category = row['Category']
        category_freq = row['Frequency']

        a = category_freq / category_totals[category]

        other_categories_freq = sum(all_word_counts[word] for cat in df['Category'].unique() if cat != category)
        b = other_categories_freq / (total_words_all_categories - category_totals[category])

        if b > 0:
            representativeness = a * np.log(1 / b)
        else:
            representativeness = 0
        word_representativeness.append([word, category, category_freq, a, b, representativeness])


    representativeness_df = pd.DataFrame(word_representativeness, columns=['Word', 'Category', 'Frequency', 'a', 'b', 'Representativeness'])

    return representativeness_df
def generate_category_tables(representativeness_df):
    category_tables = {}

    for category in representativeness_df['Category'].unique():

        category_table = representativeness_df[representativeness_df['Category'] == category]
        category_table = category_table.sort_values(by='Representativeness', ascending=False)

        category_tables[category] = category_table

    return category_tables


In [ ]:

word_freq_df = word_frequency_by_category(cleaned_df)

representativeness_df = calculate_representativeness(cleaned_df, word_freq_df)

category_tables = generate_category_tables(representativeness_df)

for category, table in category_tables.items():
    print(f"\nCategory: {category}")
    print(table.head())


Category: arts
       Word Category  Frequency         a         b  Representativeness
574     get     arts       1304  0.008543  0.011147            0.038416
921   visit     arts       1051  0.006886  0.009097            0.032362
1936     de     arts        667  0.004370  0.002043            0.027064
1094   free     arts        819  0.005366  0.007071            0.026570
87     show     arts        840  0.005503  0.008055            0.026534

Category: business
           Word  Category  Frequency         a         b  Representativeness
21630  business  business       2844  0.013986  0.004141            0.076737
21619   episode  business       1428  0.007022  0.012973            0.030512
22239       get  business       1078  0.005301  0.011193            0.023816
22240      free  business        929  0.004569  0.007100            0.022603
21811   podcast  business        969  0.004765  0.010853            0.021554

Category: educational
          Word     Category  Frequency         

In [ ]:
mannual_stop_words = ['episode','podcast','get','us','like','visit','go','youtube','also','like','instagram','use']